In [11]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
import sys
import os
import httpx

In [13]:
def make_api_request(api_endpoint, params):

    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None

In [24]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = "17c09553207c6d6e7bab10003e604aa8"
BINANCE_API_KEY = "17754325821w1b1e2khd34561hc54hh1"


def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)


def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


In [26]:
data_downloaded = get_historical_price_full_stock("AAPL")
data = data_downloaded["historical"]
data = pd.DataFrame(data)
data.columns = [x.title() for x in data.columns]  # uppercase first letter
data.drop(
    [
        "Adjclose",
        "Unadjustedvolume",
        "Change",
        "Changepercent",
        "Vwap",
        "Label",
        "Changeovertime",
    ],
    axis=1,
    inplace=True,
)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)  # date needs to be set as index!
data = data.iloc[::-1]  # to reverse the order of the dataframe
data


,Open,High,Low,Close,Volume
Date,,,,,
2018-04-02,41.9700,42.2350,41.1175,41.6700,150144764
2018-04-03,41.9100,42.1864,41.2200,42.0975,121112184
2018-04-04,41.2200,43.0025,41.1925,42.9025,138421956
2018-04-05,43.1450,43.5576,43.0200,43.2000,107576788
2018-04-06,42.7425,43.1200,42.0500,42.0950,140021160
...,...,...,...,...,...
2023-03-23,158.8300,161.5501,157.6800,158.9300,67482060
2023-03-24,158.8600,160.3400,157.8500,160.2500,59006343
2023-03-27,159.9400,160.7700,157.8700,158.2800,52390266


In [27]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as taPanda


class Ema(Strategy):
    n1 = 20
    n2 = 80
    n3 = 150

    def init(self):
        close = self.data.Close
        self.ema20 = self.I(taPanda.ema, close.s, self.n1)
        self.ema80 = self.I(taPanda.ema, close.s, self.n2)
        self.ema150 = self.I(taPanda.ema, close.s, self.n3)

    def next(self):
        price = self.data.Close
        if crossover(self.ema20, self.ema80):
            self.position.close()
            self.buy(sl=0.90 * price, tp=1.25 * price)

        elif crossover(self.ema80, self.ema20):
            self.position.close()
            self.sell(sl=1.10 * price, tp=0.75 * price)


bt = Backtest(data, Ema, cash=100000, commission=0.002, exclusive_orders=True)
output = bt.run()

output = pd.DataFrame(pd.DataFrame(output).T)

In [28]:
data = pd.DataFrame(pd.DataFrame(output).T)
data["Sizes"] = output._trades["Size"].to_string().replace("/n", ",")
data["EntryTimes"] = output._trades["EntryTime"].to_string().replace("/n", ",")
data["ExitTimes"] = output._trades["ExitTime"].to_string().replace("/n", ",")
data["Durations"] = (
    pd.DataFrame(
        [
            str(output._trades["ExitTime"][i] - output._trades["EntryTime"][i])
            for i in range(len(output._trades["ExitTime"]))
        ]
    )
    .to_string()
    .replace("/n", ",")
)
data["EntryPrices"] = output._trades["EntryPrice"].to_string().replace("/n", ",")
data["ExitPrices"] = output._trades["ExitPrice"].to_string().replace("/n", ",")
data["EntryBars"] = output._trades["EntryBar"].to_string().replace("/n", ",")
data["ExitBars"] = output._trades["ExitBar"].to_string().replace("/n", ",")
data["PnLs"] = output._trades["PnL"].to_string().replace("/n", ",")
data["ReturnPcTs"] = output._trades["ReturnPct"].to_string().replace("/n", ",")

data


In [29]:
import concurrent.futures
import glob
import os
import warnings

warnings.filterwarnings("ignore")

def run_backtests(instruments, strategy=Ema, num_threads=4):
    """Run backtests for a list of instruments using a specified strategy."""

    outputs = []
    metric = "Equity Final [$]"
    # create the output directory if it doesn't exist
    output_dir = f"output/raw/{strategy.__name__}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    def process_instrument(instrument):
        fileName = f"{output_dir}/{instrument}-*.csv"
        existingFiles = glob.glob(fileName)
        if existingFiles:
            print(f"{fileName} already exists. Skipping...")
            return None

        data = get_historical_price_full_stock(instrument)

        try:
            data = data["historical"]
            data = pd.DataFrame(data)
            data.columns = [x.title() for x in data.columns]  # uppercase first letter
            data.drop(
                [
                    "Adjclose",
                    "Unadjustedvolume",
                    "Change",
                    "Changepercent",
                    "Vwap",
                    "Label",
                    "Changeovertime",
                ],
                axis=1,
                inplace=True,
            )
            data["Date"] = pd.to_datetime(data["Date"])
            data.set_index("Date", inplace=True)  # date needs to be set as index!
            data = data.iloc[::-1]  # to reverse the order of the dataframe

            # Create a backtest for the instrument using the specified strategy
            bt = Backtest(
                data, strategy=strategy, cash=100000, commission=0.002, exclusive_orders=True
            )
            output = bt.run()
            output = pd.DataFrame(pd.DataFrame(output).T)

            # Add instrument name, strategy name, and parameters to output
            output["Instrument"] = instrument
            output["Strategy"] = strategy.__name__
            output.pop("_strategy")
            output["StrategyParameters"] = strategy.__dict__

            # Append output to list of outputs
            return output

        except Exception as e:
            print(f"Error processing {instrument}: {str(e)}")
            return None

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_instrument = {
            executor.submit(process_instrument, instrument): instrument
            for instrument in instruments
        }
        for future in concurrent.futures.as_completed(future_to_instrument):
            instrument = future_to_instrument[future]
            output = future.result()
            if output is not None:
                outputs.append(output)

                # Save output to file
                start = output["Start"].to_string().strip().split()[1]
                end = output["End"].to_string().strip().split()[1]
                fileNameOutput = (
                    f"{output_dir}/{instrument}-{start}-{end}.csv"
                )
                output.to_csv(fileNameOutput)

                # Plot and save chart to file
                fileNameChart = f"{output_dir}/{instrument}-{start}-{end}.html"
                bt.plot(filename=fileNameChart, open_browser=False)

    # Combine all the dataframes into one
    data_frame = pd.concat(outputs)

    # Save the data to a CSV file
    start = data_frame["Start"].to_string().strip().split()[1]
    end = data_frame["End"].to_string().strip().split()[1]
    fileNameOutput = f"output/{strategy.__name__}-{start}-{end}.csv"
    data_frame.to_csv(fileNameOutput)

    return outputs

In [30]:
# run_backtests(["AAPL", "MSFT", "AMZN", "GOOG", "FB", "TSLA", "NFLX", "NVDA", "PYPL", "ADBE"], Ema)

In [31]:
run_backtests(get_financial_statements_lists(), Ema)

output/raw/Ema-000004.SZ-*.csv already exists. Skipping...
output/raw/Ema-000002.SZ-*.csv already exists. Skipping...
output/raw/Ema-000005.SZ-*.csv already exists. Skipping...
output/raw/Ema-000006.SZ-*.csv already exists. Skipping...
output/raw/Ema-000001.SZ-*.csv already exists. Skipping...
output/raw/Ema-000007.SZ-*.csv already exists. Skipping...
output/raw/Ema-000008.SZ-*.csv already exists. Skipping...
output/raw/Ema-000009.SZ-*.csv already exists. Skipping...
output/raw/Ema-000011.SZ-*.csv already exists. Skipping...
output/raw/Ema-000010.SZ-*.csv already exists. Skipping...
output/raw/Ema-000012.SZ-*.csv already exists. Skipping...
output/raw/Ema-000014.SZ-*.csv already exists. Skipping...
output/raw/Ema-000019.SZ-*.csv already exists. Skipping...
output/raw/Ema-000016.SZ-*.csv already exists. Skipping...
output/raw/Ema-000017.SZ-*.csv already exists. Skipping...
output/raw/Ema-000020.KS-*.csv already exists. Skipping...
output/raw/Ema-000020.SZ-*.csv already exists. Skipping.